In [16]:
import hopsworks
import pandas as pd
import random
import os
from dotenv import load_dotenv

BACKFILL=False
load_dotenv()

True

In [17]:
def generate_flower(name, sepal_len_max, sepal_len_min, sepal_width_max, sepal_width_min, 
                    petal_len_max, petal_len_min, petal_width_max, petal_width_min):
    """
    Returns a single iris flower as a single row in a DataFrame
    """
    df = pd.DataFrame({ "sepal_length": [random.uniform(sepal_len_max, sepal_len_min)],
                       "sepal_width": [random.uniform(sepal_width_max, sepal_width_min)],
                       "petal_length": [random.uniform(petal_len_max, petal_len_min)],
                       "petal_width": [random.uniform(petal_width_max, petal_width_min)]
                      })
    df['variety'] = name
    return df


def get_random_iris_flower():
    """
    Returns a DataFrame containing one random iris flower
    """
    virginica_df = generate_flower("Virginica", 8, 5.5, 3.8, 2.2, 7, 4.5, 2.5, 1.4)
    versicolor_df = generate_flower("Versicolor", 7.5, 4.5, 3.5, 2.1, 3.1, 5.5, 1.8, 1.0)
    setosa_df =  generate_flower("Setosa", 6, 4.5, 4.5, 2.3, 1.2, 2, 0.7, 0.3)

    # randomly pick one of these 3 and write it to the featurestore
    pick_random = random.uniform(0,3)
    if pick_random >= 2:
        iris_df = virginica_df
    elif pick_random >= 1:
        iris_df = versicolor_df
    else:
        iris_df = setosa_df

    return iris_df

In [21]:
if BACKFILL==True:
    iris_df=pd.read_csv(r"C:\Users\nk424\Documents\project\serverless-ml\iris\iris.csv")
else:
    iris_df = get_random_iris_flower()
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,7.332555,2.301353,5.379912,1.829956,Virginica


In [22]:
project_name = "naveen"
api_key = os.getenv("HOPSWORKS_API_KEY")

connection = hopsworks.login(project=project_name, api_key_value=api_key)
fs = connection.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/865430
Connected. Call `.close()` to terminate connection gracefully.


In [23]:
iris_fg = fs.get_or_create_feature_group(name="iris",
                                  version=1,
                                  primary_key=["sepal_length","sepal_width","petal_length","petal_width"],
                                  description="Iris flower dataset"
                                 )
iris_fg.insert(iris_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: iris_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/865430/jobs/named/iris_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2bcb32e47a0>, None)